In [1]:
import numpy as np
import pandas as pd
import os
import torch

from ultralytics import YOLO
from tqdm import tqdm

In [133]:
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
# model = YOLO('runs/detect/train2/weights/last.pt')  # กรณีที่ train แล้วมันค้าง เอาที่เทรนไว้แล้วมา train ต่อ

In [ ]:
model.train(data='data.yaml', epochs=100, imgsz=640)

# Predict

In [2]:
model = YOLO('runs/detect/train2 (n)/weights/best.pt')  # Path to your best model

In [3]:
classes = ['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'ก',
 'ข',
 'ฃ',
 'ค',
 'ฅ',
 'ฆ',
 'ง',
 'จ',
 'ฉ',
 'ช',
 'ซ',
 'ฌ',
 'ญ',
 'ฎ',
 'ฏ',
 'ฐ',
 'ฑ',
 'ฒ',
 'ณ',
 'ด',
 'ต',
 'ถ',
 'ท',
 'ธ',
 'น',
 'บ',
 'ป',
 'ผ',
 'ฝ',
 'พ',
 'ฟ',
 'ภ',
 'ม',
 'ย',
 'ร',
 'ล',
 'ว',
 'ศ',
 'ษ',
 'ส',
 'ห',
 'ฬ',
 'อ',
 'ฮ',
 'กรุงเทพมหานคร',
 'กระบี่',
 'กาญจนบุรี',
 'กาฬสินธุ์',
 'กำแพงเพชร',
 'ขอนแก่น',
 'จันทบุรี',
 'ฉะเชิงเทรา',
 'ชลบุรี',
 'ชัยนาท',
 'ชัยภูมิ',
 'ชุมพร',
 'เชียงราย',
 'เชียงใหม่',
 'ตรัง',
 'ตราด',
 'ตาก',
 'นครนายก',
 'นครปฐม',
 'นครพนม',
 'นครราชสีมา',
 'นครศรีธรรมราช',
 'นครสวรรค์',
 'นนทบุรี',
 'นราธิวาส',
 'น่าน',
 'บึงกาฬ',
 'บุรีรัมย์',
 'ปทุมธานี',
 'ประจวบคีรีขันธ์',
 'ปราจีนบุรี',
 'ปัตตานี',
 'พระนครศรีอยุธยา',
 'พังงา',
 'พัทลุง',
 'พิจิตร',
 'พิษณุโลก',
 'เพชรบุรี',
 'เพชรบูรณ์',
 'แพร่',
 'พะเยา',
 'ภูเก็ต',
 'มหาสารคาม',
 'มุกดาหาร',
 'แม่ฮ่องสอน',
 'ยะลา',
 'ยโสธร',
 'ร้อยเอ็ด',
 'ระนอง',
 'ระยอง',
 'ราชบุรี',
 'ลพบุรี',
 'ลำปาง',
 'ลำพูน',
 'เลย',
 'ศรีสะเกษ',
 'สกลนคร',
 'สงขลา',
 'สตูล',
 'สมุทรปราการ',
 'สมุทรสงคราม',
 'สมุทรสาคร',
 'สระแก้ว',
 'สระบุรี',
 'สิงห์บุรี',
 'สุโขทัย',
 'สุพรรณบุรี',
 'สุราษฎร์ธานี',
 'สุรินทร์',
 'หนองคาย',
 'หนองบัวลำภู',
 'อ่างทอง',
 'อุดรธานี',
 'อุทัยธานี',
 'อุตรดิตถ์',
 'อุบลราชธานี',
 'อำนาจเจริญ']

## Try with one test image first

In [4]:
inputs = ['/scratch/lt900002-ai2301/heart/image/test/801_9_525_20191202161013142.jpg']
results = model(inputs)  # List of Results objects

for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmenation masks outputs
    probs = result.probs  # Class probabilities for classification outputs


0: 320x640 1 1, 1 3, 1 5, 1 9, 1 ข, 1 ช, 1 ส, 1 กรุงเทพมหานคร, 56.0ms
Speed: 3.0ms preprocess, 56.0ms inference, 275.7ms postprocess per image at shape (1, 3, 640, 640)


In [5]:
# อ่าน comment อธิบายโค้ดด้านล่าง (ตรง Predict all)

result_list = sorted(list(zip(boxes.xyxy, boxes.cls, boxes.conf)), key=lambda x: x[0][1], reverse=True)
prov_label = int(result_list[0][1])
assert(result_list[0][0][1] > 5)
remain_list = sorted(result_list[1:], key=lambda x: x[0][0])

remove_list = []
for i in range(len(remain_list)):
    rem = remain_list[i]
    if rem[1] >= 54:
        remove_list.append(i)

remain_list = [remain_list[i] for i in range(len(remain_list)) if i not in remove_list]

pair_list = []
for i in range(len(remain_list) - 1):
    pair_list.append((remain_list[i], remain_list[i+1]))

remove_list = []
for i in range(len(pair_list)):
    rem0, rem1 = pair_list[i]
    if abs(rem0[0][0] - rem1[0][0]) < 2:
        if rem0[2] > rem1[2]:
            remove_list.append(i+1)
        else:
            remove_list.append(i)

remain_list = [remain_list[i] for i in range(len(remain_list)) if i not in remove_list]
            
digit_list = [int(x[1]) for x in remain_list]
ans = ''
if prov_label == 54:
    ans += '0'
else:
    ans += '1'
    
for digit in digit_list:
    ans += classes[digit]

ans

'0ชส9153'

## Predict all

In [6]:
base_dir = '/project/lt900038-ai23tn/kaggle/im_license_plate_recognition'

In [7]:
test_df = pd.read_csv(os.path.join(base_dir, 'test.csv'))

In [8]:
new_folder = '/scratch/lt900002-ai2301/heart/image/test'
ans_list = []
error_list = []  # a list containing index and file path of files with undesirable detection result (just for error analysis)

for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    # Fead an image into the model
    
    path = os.path.join(new_folder, row['image_id'] + '.jpg')
    inputs = [path]
    with torch.no_grad():
        results = model(inputs, verbose=False)
        
    # Postprocess
    
    boxes = results[0].boxes
    # Sort list based on y-position from largest to smallest (i.e. bottommost to topmost). 
    # The bottommost one should be the box that contains the province name.
    result_list = sorted(list(zip(boxes.xyxy, boxes.cls, boxes.conf)), key=lambda x: x[0][1], reverse=True)
    
    # Note: หน้าตาของ zip(boxes.xyxy, boxes.cls, boxes.conf) จะเป็นลักษณะ
    # [([x1, y1, x2, y2], class, confidence_score), ([x1, y1, x2, y2], class, confidence_score), ....]
    
    if len(result_list) == 0:
        # If cannot detect anything
        ans_list.append('0กก5555')   # ก and 5 are the most frequently occurring alphabet and digit
        error_list.append((i, path))  # just answer 0กก5555 and move on to the next image
        continue
    if result_list[0][0][1] < 5:
        # If cannot detect any province bbox, just guess Bangkok
        prov_label = 54
        remain_list = sorted(result_list, key=lambda x: x[0][0])
        error_list.append((i, path))
    else:
        # If can detect province, save the province label and remove the bottommost bbox from the list
        prov_label = int(result_list[0][1])
        remain_list = sorted(result_list[1:], key=lambda x: x[0][0])
    
    # วิธี remove ทั้งหมดด้านล่างนี้รีบเขียนตอนท้าย เลยจะมึน ๆ หน่อย หลักคือเจอปัญหาว่ามันไม่สามารถใช้ list.remove(x) ได้โดยตรง
    # เพราะ x เป็น Tensor เลยใช้วิธีเก็บ index ของตัวที่จะ remove ไว้ แล้วมา remove ทีเดียว
    
    remove_list = []
    for i in range(len(remain_list)):
        rem = remain_list[i]
        if rem[1] >= 54:
            # If there is still any province bbox, just remove it
            remove_list.append(i)
            
    remain_list = [remain_list[i] for i in range(len(remain_list)) if i not in remove_list]
    
    # จากการทำ error analysis พบปัญหาว่าบางตัวอักษรเช่น ช โมเดลจะ detect ว่ามันเป็นทั้ง ช และ ข โดยที่ bbox จะเกือบซ้อนกัน
    # Find any pair of bboxes with similar x-position
    pair_list = []
    for i in range(len(remain_list) - 1):
        pair_list.append((remain_list[i], remain_list[i+1]))
        
    remove_list = []
    for i in range(len(pair_list)):
        p0, p1 = pair_list[i]
        if abs(p0[0][0] - p1[0][0]) < 2:
            # If the bboxes are too close, remove one with a lower confidence score
            if p0[2] > p1[2]:
                remove_list.append(i+1)
            else:
                remove_list.append(i)
                
    remain_list = [remain_list[i] for i in range(len(remain_list)) if i not in remove_list]
    
    # Convert to the answer format
    digit_list = [int(x[1]) for x in remain_list]
    ans = ''
    if prov_label == 54:
        ans += '0'
    else:
        ans += '1'
    
    for digit in digit_list:
        ans += classes[digit]

    ans_list.append(ans)

100%|██████████| 1991/1991 [00:21<00:00, 91.34it/s]


In [10]:
test_df['label'] = ans_list
test_df

,image_id,x1,y1,x2,y2,label
0,801_9_1445_20191202102441298,333.014354,205.741627,394.497608,238.277512,040674
1,801_9_1_20191202070338305,293.574297,201.867470,358.634538,235.200803,05กฒ4901
2,801_9_42_20191202071224391,306.425703,204.277108,362.248996,227.168675,1กฉ9661
3,801_9_68_20191202201851852,332.530120,199.056225,397.991968,228.775100,0จห464
4,801_9_580_20191202080915065,312.466844,160.477454,378.779841,193.103448,0ฌธ5479
...,...,...,...,...,...,...
1986,801_9_1302_20191202095231132,343.902439,209.829268,418.292683,243.365854,0ฌย5620
1987,801_9_708_20191202065656188,345.272727,193.636364,413.818182,224.727273,03กข9152
1988,801_9_190_20191202124956605,333.567901,201.925926,395.296296,230.938272,05กฎ1222
1989,801_9_59_20191202104142666,336.546185,213.112450,401.606426,238.413655,0ฒญ3489


In [11]:
sub_df = pd.read_csv(os.path.join(base_dir, 'submission.csv'))
sub_df

,image_id,label
0,801_9_1445_20191202102441298,0ฬ-0674
1,801_9_1_20191202070338305,05กฒ4901
2,801_9_42_20191202071224391,1กฉ9661
3,801_9_580_20191202080915065,NaN
4,801_9_646_20191202081630216,NaN
...,...,...
1695,801_9_85_20191202172945268,NaN
1696,801_9_39_20191202165306745,NaN
1697,801_9_525_20191202161013142,NaN
1698,801_9_466_20191202154226314,NaN


In [13]:
merged_sub_df = sub_df.merge(test_df[['image_id', 'label']], how='left', left_on='image_id', right_on='image_id')
merged_sub_df

,image_id,label_x,label_y
0,801_9_1445_20191202102441298,0ฬ-0674,040674
1,801_9_1_20191202070338305,05กฒ4901,05กฒ4901
2,801_9_42_20191202071224391,1กฉ9661,1กฉ9661
3,801_9_580_20191202080915065,NaN,0ฌธ5479
4,801_9_646_20191202081630216,NaN,05กฒ576
...,...,...,...
1696,801_9_85_20191202172945268,NaN,03กท1727
1697,801_9_39_20191202165306745,NaN,09กข9849
1698,801_9_525_20191202161013142,NaN,0ชส9153
1699,801_9_466_20191202154226314,NaN,01ฒอ695


In [17]:
label_list = list(merged_sub_df['label_x'])[:3] + list(merged_sub_df['label_y'])[3:]
merged_sub_df['label'] = label_list

In [18]:
merged_sub_df

,image_id,label_x,label_y,label
0,801_9_1445_20191202102441298,0ฬ-0674,040674,0ฬ-0674
1,801_9_1_20191202070338305,05กฒ4901,05กฒ4901,05กฒ4901
2,801_9_42_20191202071224391,1กฉ9661,1กฉ9661,1กฉ9661
3,801_9_580_20191202080915065,NaN,0ฌธ5479,0ฌธ5479
4,801_9_646_20191202081630216,NaN,05กฒ576,05กฒ576
...,...,...,...,...
1696,801_9_85_20191202172945268,NaN,03กท1727,03กท1727
1697,801_9_39_20191202165306745,NaN,09กข9849,09กข9849
1698,801_9_525_20191202161013142,NaN,0ชส9153,0ชส9153
1699,801_9_466_20191202154226314,NaN,01ฒอ695,01ฒอ695


In [19]:
merged_sub_df[merged_sub_df['image_id'] == '801_9_1044_20191202091027326']

,image_id,label_x,label_y,label
1197,801_9_1044_20191202091027326,NaN,08กฐ884,08กฐ884
1198,801_9_1044_20191202091027326,NaN,08กฐ884,08กฐ884


In [20]:
merged_sub_df = merged_sub_df.drop_duplicates()

In [21]:
# Check if the order and number of ids are correct
(np.array(merged_sub_df['image_id']) == np.array(sub_df['image_id'])).all()

True

In [22]:
merged_sub_df[['image_id', 'label']].to_csv('submission.csv', index=False)

## Error analysis

In [227]:
ans_list

['040674',
 '05กฒ4901',
 '1กฉ9661',
 '0จห464',
 '0ฌธ5479',
 '05กฒ576',
 '06กษ4549',
 '02กย369',
 '0ขท755',
 '0พม73',
 '08กต8042',
 '0ญร5186',
 '07กก4549',
 '0ฮษ645',
 '05255125',
 '05กล8729',
 '05กล5235',
 '0กข6804',
 '03กบ6161',
 '01ฒฉ728',
 '0ฎท396',
 '02กต6678',
 '05กภ8896',
 '03กข7554',
 '07กต6780',
 '0ฆฮ9182',
 '06กม160',
 '0ญฒ618',
 '0ฒม5139',
 '08กผ4767',
 '0ฏณ9499',
 '03กท4516',
 '03กส3414',
 '07กท6998',
 '04กษ5046',
 '0ฎภ794',
 '06กฉ5160',
 '07กว2105',
 '06กฒ3901',
 '18กค685',
 '04กค8159',
 '07กร251',
 '0สณ75017',
 '0ญง3289',
 '05กค4032',
 '05กฮ1715',
 '06กผ2439',
 '0ขม6046',
 '08กฌ927',
 '0ศห8668',
 '07กห7884',
 '04กฆ9980',
 '0ญร613',
 '06กต2574',
 '1กง789',
 '07กญ2441',
 '0ฉร9955',
 '05กต6070',
 '03กผ59',
 '0ฎบ1112',
 '06กช9612',
 '05กญ4434',
 '0ศจ1331',
 '06ก9431',
 '01กฒ7030',
 '04กฮ2214',
 '0ญว1717',
 '07กว4361',
 '07กฬ2837',
 '0ชม9447',
 '02ฒท889',
 '08ก374',
 '0ฒณ2734',
 '02กพ5495',
 '08กณ8325',
 '0ญท236',
 '04กข340',
 '02กข3384',
 '07กศ205',
 '0กท7819',
 '01กล217',
 '0

In [ ]:
fix bbox that ซ้อนกัน เช่น ข ช

In [24]:
ans_list[:3]

['040674', '05กฒ4901', '1กฉ9661']

In [25]:
sub_list = list(sub_df['label'][:3]) + ans_list[3:]

In [27]:
len(sub_list)

1991

In [ ]:
sub_df['label'] = 

In [ ]:
for error in error_list:
    print(ans_list[error[0]])

In [91]:
path

'/scratch/lt900002-ai2301/heart/image/test/801_9_5_20191202000459540.jpg'

In [92]:
res_list

[]

ก    1293
5    1038
1     996
8     963
2     962
9     927
4     924
6     856
7     842
3     816
0     457
ร     262
ฒ     204
ม     170
ค     158
พ     152
ห     148
น     145
ท     138
ข     135
ฆ     125
ง     125
ฎ     119
ญ     118
ฮ     110
ว      91
ษ      90
ณ      87
า      86
ุ      84
ต      82
เ      81
บ      81
ฌ      81
ฏ      81
ส      80
ช      80
ศ      78
ธ      75
ย      72
ฉ      71
ภ      69
ถ      64
ล      63
อ      63
ฐ      61
จ      56
ผ      48
ฬ      34
ี      20
ด      14
ะ      14
ป      11
ั       7
์       7
ฟ       4